In [9]:
import os
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# ===============================
# 1. Khởi tạo Spark + Delta
# ===============================
builder = (
    SparkSession.builder
        .appName("Bronze_Ingestion_Layer")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [10]:
# ===============================
# 2. Cấu hình đường dẫn
# ===============================
base_path = "/home/jovyan/work"
landing_zone = f"{base_path}/data_landing"
bronze_zone  = f"{base_path}/data_lake/bronze"

print(f"🚀 [BRONZE] Bắt đầu nạp dữ liệu từ: {landing_zone}")

🚀 [BRONZE] Bắt đầu nạp dữ liệu từ: /home/jovyan/work/data_landing


In [12]:
# ===============================
# 3. Hàm nạp JSON → Delta
# ===============================
def ingest_json_to_delta(file_name, bronze_name):
    try:
        print(f"\n⏳ Đang đọc file {file_name}...")
        df = spark.read.json(f"{landing_zone}/{file_name}")
        save_path = f"{bronze_zone}/{bronze_name}"
        df.write.format("delta").mode("overwrite").save(save_path)
        print(f"✅ [SUCCESS] Đã lưu {file_name} → {save_path}")
        print("   📌 Schema:")
        df.printSchema()
    except Exception as e:
        print(f"❌ [ERROR] Lỗi khi xử lý {file_name}: {e}")

# ===============================
# 4. Ingest OTTO Train & Test
# ===============================
ingest_json_to_delta("train.jsonl", "otto_train")
ingest_json_to_delta("test.jsonl", "otto_test")

# ===============================
# 5. (Tùy chọn) Amazon Reviews
# ===============================
# ingest_json_to_delta("amazon_real_100k.jsonl", "amazon_reviews")

# ===============================
# 6. Kết thúc
# ===============================
print("\n🏁 [FINISH] Hoàn tất Bronze Layer.")
spark.stop()


⏳ Đang đọc file train.jsonl...
❌ [ERROR] Lỗi khi xử lý train.jsonl: An error occurred while calling o117.read.
: java.lang.NoClassDefFoundError: scala/collection/SeqOps
	at io.delta.sql.parser.DeltaSqlParser.<init>(DeltaSqlParser.scala:78)
	at io.delta.sql.DeltaSparkSessionExtension.$anonfun$apply$1(DeltaSparkSessionExtension.scala:85)
	at org.apache.spark.sql.SparkSessionExtensions.$anonfun$buildParser$1(SparkSessionExtensions.scala:308)
	at scala.collection.IndexedSeqOptimized.foldLeft(IndexedSeqOptimized.scala:60)
	at scala.collection.IndexedSeqOptimized.foldLeft$(IndexedSeqOptimized.scala:68)
	at scala.collection.mutable.ArrayBuffer.foldLeft(ArrayBuffer.scala:49)
	at org.apache.spark.sql.SparkSessionExtensions.buildParser(SparkSessionExtensions.scala:307)
	at org.apache.spark.sql.internal.BaseSessionStateBuilder.sqlParser$lzycompute(BaseSessionStateBuilder.scala:138)
	at org.apache.spark.sql.internal.BaseSessionStateBuilder.sqlParser(BaseSessionStateBuilder.scala:137)
	at org.apac